In [ ]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
notebook_path = os.path.abspath(".")

In [ ]:
covidDataJSON = requests.request("GET", "https://covidtracking.com/api/states/daily")

In [ ]:
df_covid = pd.read_json(covidDataJSON.text)
df_new_york = df_covid.loc[df_covid['state'] == "NY"].sort_values("date").set_index("date")
df_new_york["s_date"] = df_new_york.index.astype('str')
df_new_york["s_date"] = df_new_york.s_date.str[0:4] + "-" + df_new_york.s_date.str[4:6] + "-" + df_new_york.s_date.str[6:8]

In [ ]:
ax = plt.gca()
df_new_york.plot(kind="line",y="positive", x="s_date", color="red", ax=ax)
ax.set_xlabel("Date")
ax.set_ylabel("Positive Tests")
ax.get_legend().remove()
plt.yscale("log")
plt_ny_positives = plt.gcf()
plt_ny_positives.set_size_inches(20, 10)
plt_ny_positives.savefig("log-history.png")

In [ ]:
df_pos_pct_change = df_new_york["positive"].pct_change().to_frame().reset_index().set_index("date")

In [ ]:
ax = plt.gca()
df_pos_pct_change["s_date"] = df_new_york["s_date"]
df_pos_pct_change["percent"] = df_pos_pct_change["positive"] * 100
df_pos_pct_change.plot(kind="line",y="percent", x="s_date", color="red", ax=ax)
ax.set_xlabel("Date")
ax.set_ylabel("% Increase in Positives")
plt_ny_pct_change = plt.gcf()
plt_ny_pct_change.set_size_inches(20, 10)

In [ ]:
df_pos_pct_change["date"] = df_pos_pct_change.index

In [ ]:
sns.regplot(x='date',y='percent',data=df_pos_pct_change, fit_reg=True)
plt_ny_pct_change_best_fit = plt.gcf()
plt_ny_pct_change_best_fit.set_size_inches(20, 10)

In [ ]:
df_pos_pct_change